In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time # for sleep time
import csv
import re
import functools
import datetime #for csv file title
from bs4 import BeautifulSoup as bs
import requests

house_title_list = []

driver = webdriver.Chrome(r'/Users/msellamia/airbnb/chromedriver')

driver.get('https://www.airbnb.com/s/Toronto--ON--Canada/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&search_type=filter_change&adults=1&place_id=ChIJpTvG15DL1IkRd8S0KlBVNTI&screen_size=large&hide_dates_and_guests_filters=false')

now = datetime.datetime.now() #for csv file title

#For csv file names
def reduce_concat(x, sep=""):
    return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

def paste(*lists, sep=" ", collapse=None):
    result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
    if collapse is not None:
        return reduce_concat(result, sep=collapse)
    return list(result)

#Open CSV Writer
filename = paste(['airBnb_To'], [now.year], ['-'], [now.month], ['-'], [now.day],['.csv'], sep='')
filename = ''.join(filename)
csv_file = open(filename, 'w', encoding='utf-8',newline = '')
writer = csv.writer(csv_file)

index = 1

house_all_links = []

while True:
    
    if index > 17:
        break
    try:
        print("Getting list of links from Page " + str(index))
        index = index + 1
        #driver.maximize_window()
        time.sleep(6) # give new page time to load
                
        house_list = (driver.find_elements_by_xpath('//a[@class="_1wtrbgoo"]'))

        house_links = [house.get_attribute('href') for house in house_list ] 
        
        house_all_links.extend(house_links)
  
        # Locate the next button on the page.
        wait_button = WebDriverWait(driver, 10)
        
        next_button = wait_button.until(EC.element_to_be_clickable((By.XPATH,'//li[@class="_r4n1gzb"]')))
        
        next_button.click()

        
    except Exception as e:
        print(e)
        continue
        
print(house_all_links)
for link in house_all_links: 
    page = requests.get(link)
    soup = bs(page.text, "lxml")
    match = soup.find_all('a', href=True)
    link_list = []
    for i in match:
        link = i["href"]
        link_list.append(link)  
    for url in link_list[2:]:
        url = ("https://www.airbnb.ca"+str(url))
        airbnb_dict = {}
        driver.get(url)
        time.sleep(7)

        try:
            house_title = driver.find_element_by_xpath('//h1/span[@class="_18hrqvin"]').text
        except:
            house_title = None
        try:
            house_location =  driver.find_element_by_xpath('//div[@class="_czm8crp"]').text
        except:
            house_location = None

        try:
            price = driver.find_element_by_xpath('//span/span[@class="_doc79r"]').text
        except:
            price = None        

        try:
            no_of_reviews = driver.find_element_by_xpath('//div[@class="_14hveccc"]').text
        except:
            no_of_reviews = None  

        try:
            rating = driver.find_element_by_xpath('//div[@class="_10za72m2"]').text

        except:
            rating = None

        try:
            max_occupancy = driver.find_elements_by_xpath('//div/div[@class="_36rlri"]/div[@class="_czm8crp"]')
            house_detail = [detail.text for detail in max_occupancy ]

        except:

            house_detail = None
        if house_title not in house_title_list:
            house_title_list.append(house_title)
            print ([house_title, house_location, price, no_of_reviews, rating, house_detail])
            
            airbnb_dict['house_title'] = house_title
            airbnb_dict['house_locatione'] = house_location
            airbnb_dict['price'] = price
            airbnb_dict['no_of_reviews'] = no_of_reviews
            airbnb_dict['rating'] = rating
            airbnb_dict['house_detail'] = house_detail
            writer.writerow(airbnb_dict.values())

csv_file.close()
driver.close()


Getting list of links from Page 1
Getting list of links from Page 2
Getting list of links from Page 3
Getting list of links from Page 4
Getting list of links from Page 5
Getting list of links from Page 6
Getting list of links from Page 7
Getting list of links from Page 8
Getting list of links from Page 9
Getting list of links from Page 10
Getting list of links from Page 11
Getting list of links from Page 12
Getting list of links from Page 13
Getting list of links from Page 14
Getting list of links from Page 15
Getting list of links from Page 16
Getting list of links from Page 17
['https://www.airbnb.com/s/Toronto--ON--Canada/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&search_type=pagination&place_id=ChIJpTvG15DL1IkRd8S0KlBVNTI&screen_size=large&hide_dates_and_guests_filters=false&adults=1&s_tag=03EAP-iZ', 'https://www.airbnb.com/s/Toronto--ON--Canada/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&search_

['Studio RED', 'Toronto', '$61', '345 reviews', '4.29', ['6 guests', 'Studio', '3 beds', '1 bath', '\U000f0004\n7975 Reviews', '\U000f0019\nVerified']]
["Luxury Suite in Toronto's Entertainment District", 'Toronto', '$120', '756 reviews', '4.89', ['3 guests', '1 bedroom', '2 beds', '1 bath', '\U000f0004\n756 Reviews', '\U000f1101\n2 References', '\U000f0019\nVerified']]
['Japan Inspired Studio Apartment', 'Toronto', '$76', '391 reviews', '4.37', ['4 guests', 'Studio', '2 beds', '1 bath', '\U000f0004\n7975 Reviews', '\U000f0019\nVerified']]
['Downtown Entertainment District! Great City View!', 'Toronto', '$93', '433 reviews', '4.64', ['4 guests', '1 bedroom', '2 beds', '1 bath', '\U000f0004\n647 Reviews', '\U000f0019\nVerified']]
['Retro 1 bedroom apartment at downtown core', 'Toronto', '$75', '371 reviews', '4.34', ['4 guests', '1 bedroom', '2 beds', '1 bath', '\U000f0004\n7975 Reviews', '\U000f0019\nVerified']]
['High Floor |Balcony | Lake View | 100% Walk | PATH', 'Toronto', '$99', '

['Newly Furnished Condo w/Balcony across CN Tower', 'Toronto', '$115', '376 reviews', '4.94', ['6 guests', '1 bedroom', '3 beds', '1 bath', '\U000f0004\n1049 Reviews', '\U000f0019\nVerified']]
['Downtown hotel style studio w/parking/ 6 ppl max', 'Toronto', '$40', '258 reviews', '4.34', ['6 guests', '1 bedroom', '3 beds', '1 bath', '\U000f0004\n1884 Reviews', '\U000f0019\nVerified']]
['Big bright private Bloordale room w/double futon', 'Toronto', '$34', '134 reviews', '4.84', ['2 guests', '1 bedroom', '1 bed', '1 shared bath', '\U000f0004\n256 Reviews', '\U000f0019\nVerified']]
['Stylish Downtown Lovely Room', 'Toronto', '$50', '435 reviews', '4.51', ['2 guests', '1 bedroom', '1 bed', '2 shared baths', '\U000f0004\n2235 Reviews', '\U000f0019\nVerified']]
['Room Snuggy Yellow - Little Italy', 'Toronto', '$40', '258 reviews', '4.37', ['2 guests', '1 bedroom', '1 bed', '1 shared bath', '\U000f0004\n7975 Reviews', '\U000f0019\nVerified']]
['LUXURY! Prime Location+CN Tower View! FREE PARKING

['Beautiful Condo at 300 Front St', 'Toronto', '$125', '419 reviews', '4.93', ['3 guests', '1 bedroom', '1 bed', '1 bath', '\U000f0004\n419 Reviews']]
['Furnished Condo w/Balcony Very Close To CN Tower!!', 'Toronto', '$110', '301 reviews', '4.91', ['4 guests', '1 bedroom', '2 beds', '1 bath', '\U000f0004\n1049 Reviews', '\U000f0019\nVerified']]
['60th+ floor sub penthouse lake view condo', 'Toronto', '$107', '177 reviews', '4.89', ['3 guests', '1 bedroom', '2 beds', '1 bath', '\U000f0004\n177 Reviews', '\U000f0019\nVerified']]
['SPECTACULAR DOWNTOWN CONDO W/ STUNNING CITY VIEWS', 'Toronto', '$109', '124 reviews', '4.90', ['5 guests', '1 bedroom', '2 beds', '1 bath', '\U000f0004\n124 Reviews', '\U000f0019\nVerified']]
['Paradise Downtown Toronto', 'Toronto', '$79', '407 reviews', '4.70', ['4 guests', '1 bedroom', '2 beds', '1 bath', '\U000f0004\n944 Reviews', '\U000f0019\nVerified']]


ConnectionError: HTTPSConnectionPool(host='www.airbnb.com', port=443): Max retries exceeded with url: /s/Toronto--ON--Canada/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&search_type=pagination&place_id=ChIJpTvG15DL1IkRd8S0KlBVNTI&screen_size=large&hide_dates_and_guests_filters=false&s_tag=03EAP-iZ&section_offset=4&items_offset=126&adults=1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1162282e8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))